In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
import tensorflow as tf
import numpy as np
import tfcoreml
from tensorflow.python.tools.freeze_graph import freeze_graph

In [0]:
with tf.name_scope("net"):
    input = tf.placeholder(tf.float32, shape=(None, 28, 28, 1), name="input")
    conv1 = tf.layers.conv2d(input, 20, 5, activation=tf.nn.relu)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
    conv2 = tf.layers.conv2d(conv1, 32, 3, activation=tf.nn.relu)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    flatten = tf.layers.flatten(conv2)
    fc1 = tf.layers.dense(flatten, 256, activation=tf.nn.relu)
    output = tf.layers.dense(fc1, 10)
    result = tf.argmax(output, axis=1, name="result")

In [0]:
with tf.name_scope("train"):
    labels = tf.placeholder(tf.int32, shape=(None))
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=labels))
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [0]:
with tf.name_scope("eval"):
    # ArgMax of 1-sample input as 4D array, to circumvent tfcoreml argmax bug
    result_alt = tf.argmax(tf.reshape(output, (1, 1, 1, 10)), axis = 3, name = "result_alt")

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
for i in range(10):
    avg_loss = 0
    for j in range(1000):
        _, curr_loss = sess.run((train, loss), feed_dict={
            input: np.reshape(mnist.train.images[55 * j: 55 * (j + 1)], (-1, 28, 28, 1)),
            labels: mnist.train.labels[55 * j: 55 * (j + 1)]})
        avg_loss += curr_loss / 1000
    print(avg_loss)

0.1461365261419814
0.08100989671205394
0.07519070401717538
0.07602644162422262
0.06773538513146553
0.06052951895418741
0.05900409564076155
0.0678543020120166
0.06147829113705083
0.06924019073319937


In [0]:
test_input = np.reshape(mnist.test.images, (-1, 28, 28, 1))
total_correct = 0
for i in range(10):
  result_output = sess.run(result, feed_dict={input: test_input[1000 * i: 1000 * (i + 1)]})
  total_correct += sum(result_output == mnist.test.labels[1000 * i: 1000 * (i + 1)])

total_correct / mnist.test.labels.shape[0]

0.9856

In [0]:
saver = tf.train.Saver()
saver.save(sess, "./model/graph")
tf.train.write_graph(sess.graph, "./model/", "graph.pbtxt")

'./model/graph.pbtxt'

In [0]:
with tf.Graph().as_default():
    freeze_graph(input_graph = "./model/graph.pbtxt",
                 input_checkpoint = "./model/graph",
                 input_saver = "",
                 input_binary = False,
                 output_node_names = "eval/result_alt",
                 output_graph = "model.pb",
                 restore_op_name = "save/restore_all",
                 filename_tensor_name = "save/Const:0",
                 clear_devices = True,
                 initializer_nodes = "")

INFO:tensorflow:Restoring parameters from ./model/graph
INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.


In [0]:
model = tfcoreml.convert(tf_model_path = "model.pb",
                         mlmodel_path = "model.mlmodel",
                         output_feature_names = ("eval/result_alt:0",),
                         input_name_shape_dict = {"net/input:0": (1, 28, 28, 1)})

Shapes not found for 15 tensors. Executing graph to determine shapes. 
Automatic shape interpretation succeeded for input blob net/input:0
4/42: Converting op name: dense_1/bias/read ( type:  Identity )
6/42: Converting op name: dense_1/kernel/read ( type:  Identity )
8/42: Converting op name: dense/bias/read ( type:  Identity )
10/42: Converting op name: dense/kernel/read ( type:  Identity )
16/42: Converting op name: conv2d_1/bias/read ( type:  Identity )
18/42: Converting op name: conv2d_1/kernel/read ( type:  Identity )
20/42: Converting op name: conv2d/bias/read ( type:  Identity )
22/42: Converting op name: conv2d/kernel/read ( type:  Identity )
23/42: Converting op name: net/input ( type:  Placeholder )
Skipping name of placeholder
24/42: Converting op name: net/conv2d/Conv2D ( type:  Conv2D )
25/42: Converting op name: net/conv2d/BiasAdd ( type:  BiasAdd )
26/42: Converting op name: net/conv2d/Relu ( type:  Relu )
27/42: Converting op name: net/max_pooling2d/MaxPool ( type:  Ma

In [0]:
output_mlmodel = tuple(map(lambda data: int(model.predict({"net__input__0": data})["eval__result_alt__0"]),
                           np.reshape(mnist.test.images, (-1, 1, 28, 28))))
print("Correct: %.2f" % (sum(output_mlmodel == mnist.test.labels) / mnist.test.num_examples * 100) + "%")

Exception: ignored

In [3]:
!ls

data  datalab  model  model.mlmodel  model.pb  tf-coreml


In [0]:
from google.colab import files
files.download('model.mlmodel')